#### Data_Hälso_förjupning.

In [250]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression


np.random.seed(42)


#### Delar från Del1

In [251]:
def visa_statistik(df):
    kolumner = ["age", "systolic_pb", "cholesterol", "weitht" ]
    for kol in kolumner:
        print(f"{kol}: medel={df[kol].mean():.f}, median={df[kol].median():.f}")

kolumner = ["age", "weight", "height", "systolic_bp", "cholesterol"]
stats_df = pd.DataFrame(columns=["mean", "median", "min", "max"], index=kolumner)

for col in kolumner:
    col_vals = df[col].dropna()
    stats_df.loc[col] = [np.mean(col_vals), np.median(col_vals), np.min(col_vals), np.max(col_vals)]

stats_df
stats_df.to_csv("statistics_health.csv")
print("statistics_health.csv")

def skapa_grafer(df):
    fig, axs = plt.Subplots(2, 2, figsize=(12, 8))

    num_kol = ["age", "weight", "systolic_pb", "cholesterol" ]
    sns.heatmap(df[num_kol].corr(),annot=True, ax=axs[0,0])

    axs[0,1].scatter(df["age"], df["systolic_pb"], alpha=0.5)
    axs[0,1].set_title("Ålder vs Blodtryck")
    axs[0,1].boxplot([df[df["sex"]=="M"]["systolick_pb"],
                    df[df["sex"]=="F"]["systolic_pb"]],
                    labels=["Män", "Kvinnor"])
    
    sjukdom_per_kön = df.groupby("sex")["disease"].mean() * 100
    axs[1,1].bar("sjuk_per_kön.index, sjuk_per_kön.values")
    
    plt.tight_layout()
    plt.show()
    skapa_grafer
    

statistics_health.csv


#### Klasser:

In [252]:
class HealsAnalyz:

    def __init__(self, data):
        self.data = data

    def sammanfattning(self):
        return{
            "antal": len(self.data),
            "antal_sjuk": self.data["disease"].mean() * 100,
            "medelålder": self.data["age"].mean()
        }
    
    def regression(self, x_kol, y_kol):
        x = self.data[[x_kol]] if isinstance(x_kol, str) else self.data[x_kol]
        y =self.data[y_kol]

        
        model = LinearRegression()
        model.fit(x, y)

        return {
            "r2": model.score(x, y),
            "koeff": model.coef_,
            "intercept": model.intercept_
        }
   
analysator = HealsAnalyz(df)
print("sammanfattning:")
for nyckel, värde in analysator.sammanfattning().items():
        print(f"{nyckel}: {värde}")


sammanfattning:
antal: 800
antal_sjuk: 5.875
medelålder: 49.42625


#### Linjer algebra - Regression:

In [253]:
print("\nREGRESSION ANALYS:")

reg1 = analysator.regression('age', 'systolic_bp')
print(f"Ålder -> Blodtryck: R² = {reg1['r2']:.3f}")

reg2 = analysator.regression(['age', 'weight'], 'systolic_bp')
print(f"Ålder + Vikt -> Blodtryck: R² = {reg2['r2']:.3f}")



REGRESSION ANALYS:
Ålder -> Blodtryck: R² = 0.369
Ålder + Vikt -> Blodtryck: R² = 0.405


#### Linjer Algebra - PCA:

In [263]:
print("\nPCA ANALYS:")
from sklearn.decomposition import PCA

num_data = df[['age', 'syslotic_bp', 'cholesterol', 'wight']].dropna
pca =PCA(n_components=2)
pca_result = pca.fit_transform(num_data)

print(f"PC1 förlkarar {pca.explained_variance_ratio_[0]*100:.f}% av varjansen")
print(f"PC2 förlkarar {pca.explained_variance_ratio_[1]*100:.f}% av varjansen")

plt.Figure(figsize=(7.5))
plt.scatter(pca_result[:,0], pca_result[:,1], alpha=0.5)
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.title('PCA: PCA1 vs PCA2')
plt.show


PCA ANALYS:


KeyError: "['syslotic_bp', 'wight'] not in index"

Utökad Analys:

In [260]:
print("\nUTÖKAD ANALYS:")
df['Äldersgräns'] =pd.cut(df['age'], bins=[0, 40, 60, 100], labels=['<40', '40-60', '60+'])
sjuk_per_ålder = df.groupby('åldersgrupp')['disease'].mean() * 100
print("sjukdomrisk per ålder:")

for ålder, risk in sjuk_per_ålder.items():
    print(f" {ålder}: {risk:.1f}%")

    bp_män = df[df['sex']=='M']['systolic_bp'].mean()
    bp_kvinnor = df[df['sex']=='F']['systolic_bp'].mean()
    print(f'\nBlodtryck: män={bp_män:.1f}, kvinnor={bp_kvinnor:.1f}')

    fig,(ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))


    ax1.bar(sjuk_per_ålder.index.astype(str), sjuk_per_ålder.values)
    ax1.set_title('sjukdomsrisk per ålder')

    ax2.boxplot([df[df['diesease']==0]['vikt'], df[df['disease']==1]['vikt']], labels= ['Frisk', 'Sjuk'])
    ax2.set_title('Vikt för sjuka och friska')

    plt.tight_layout()
    plt.show()


UTÖKAD ANALYS:


KeyError: 'åldersgrupp'